<a href="https://colab.research.google.com/github/kyokicchi/ggl_an_codes/blob/master/trainNNmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from google.colab import drive
from google.colab import files
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
from keras import models
from keras import layers

from keras.models import load_model


Using TensorFlow backend.


In [2]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
%cd ../content/gdrive/My Drive/projects/ggl_an/
%ls

/content/gdrive/My Drive/projects/ggl_an
model_LGB.sav  user_train.csv  user_train_n.csv


In [0]:
s_data = 'user_train_n.csv'
s_col_tgt = 'transactionRevenue'
l_col_drop = [s_col_tgt]
filename = 'model_NN.h5'

In [5]:
%%time
df_train = pd.read_csv(s_data, index_col = 0)

<string>:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 3.35 s, sys: 441 ms, total: 3.79 s
Wall time: 5.7 s


In [6]:
%%time
df_y = df_train[s_col_tgt]
df_x = df_train.drop(l_col_drop, axis=1)

CPU times: user 27.8 ms, sys: 70.1 ms, total: 97.9 ms
Wall time: 111 ms


In [0]:
from keras import optimizers


def build_model(N_col):
    model = models.Sequential()
    model.add(layers.Dense(120, activation='relu', input_shape=(N_col,)))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(60, activation='relu'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(1))
    model.compile(optimizer=optimizers.adam(lr=0.01), loss='mse', metrics=['mae'])
    return model

In [12]:
%%time
epochs = 100
batch_size = df_x.shape[0] // 4
validation_split = 0.1

model = build_model(df_x.shape[1])

history = model.fit(df_x,
                    df_y,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_split = validation_split,
                    verbose=1)

Train on 642750 samples, validate on 71417 samples
Epoch 1/100
642750/642750 [==============================] - 1s 2us/step - loss: 8.4628 - mean_absolute_error: 0.6207 - val_loss: 8.4997 - val_mean_absolute_error: 0.8899
Epoch 2/100
642750/642750 [==============================] - 1s 1us/step - loss: 8.2293 - mean_absolute_error: 0.7066 - val_loss: 8.2374 - val_mean_absolute_error: 0.4715
Epoch 3/100
642750/642750 [==============================] - 1s 1us/step - loss: 8.0049 - mean_absolute_error: 0.5851 - val_loss: 8.0228 - val_mean_absolute_error: 0.5977
Epoch 4/100
642750/642750 [==============================] - 1s 1us/step - loss: 7.8245 - mean_absolute_error: 0.5832 - val_loss: 7.7889 - val_mean_absolute_error: 0.5924
Epoch 5/100
642750/642750 [==============================] - 1s 1us/step - loss: 7.5924 - mean_absolute_error: 0.5745 - val_loss: 7.5215 - val_mean_absolute_error: 0.5179
Epoch 6/100
642750/642750 [==============================] - 1s 1us/step - loss: 7.3317 - mean

In [0]:
#history.history

In [14]:
%%time
model.save(filename)

#files.download(filename)

CPU times: user 77 ms, sys: 14 ms, total: 91 ms
Wall time: 107 ms
